In [1]:
import wrds
file_key = open("../dev/archive/wrds_username.txt","r")
USERNAME = file_key.read()
file_key.close()

db = wrds.Connection(wrds_username=USERNAME)
#db.create_pgpass_file()

Loading library list...
Done


In [2]:
import pandas as pd
import numpy as np
from datetime import date

In [3]:
libs = db.list_libraries()
libs_crsp = [item for item in libs if item.startswith('crsp')]
libs_crsp

['crsp',
 'crsp_a_ccm',
 'crsp_a_indexes',
 'crsp_a_stock',
 'crsp_a_stock62',
 'crsp_a_stock62_new',
 'crsp_a_stock_new',
 'crsp_a_treasuries',
 'crsp_a_ziman',
 'crsp_m_ccm',
 'crsp_m_indexes',
 'crsp_m_stock',
 'crsp_m_stock62',
 'crsp_m_treasuries',
 'crsp_m_ziman',
 'crsp_q_ccm',
 'crsp_q_indexes',
 'crsp_q_indexhist',
 'crsp_q_mutualfunds',
 'crsp_q_mutualfunds_new',
 'crsp_q_stock',
 'crsp_q_stock62',
 'crsp_q_treasuries',
 'crsp_q_ziman',
 'crspa',
 'crspm',
 'crspq',
 'crspsamp']

In [4]:
db.list_tables('crsp_m_treasuries')

['tfz_dly',
 'tfz_dly_cd',
 'tfz_dly_cpi',
 'tfz_dly_ft',
 'tfz_dly_rf2',
 'tfz_dly_ts2',
 'tfz_idx',
 'tfz_iss',
 'tfz_mast',
 'tfz_mth',
 'tfz_mth_bp',
 'tfz_mth_cd',
 'tfz_mth_cpi',
 'tfz_mth_fb',
 'tfz_mth_ft',
 'tfz_mth_rf',
 'tfz_mth_rf2',
 'tfz_mth_ts',
 'tfz_mth_ts2',
 'tfz_pay']

In [5]:
data = db.get_table(library='crsp_m_treasuries', table='tfz_mth_fb')
yields = data.pivot_table(values='tmytm',index='mcaldt',columns='kytreasnox') / 100
prices = data.pivot_table(values='tmnomprc',index='mcaldt',columns='kytreasnox') / 100

In [6]:
prices.columns = [1, 2, 3, 4, 5]
prices.index.name = 'date'
prices.columns.name = 'maturity'
prices

maturity,1,2,3,4,5
date,,,,,
1952-06-30,0.981778,0.962531,0.942467,0.926341,0.904183
1952-07-31,0.980948,0.960315,0.939312,0.923089,0.898516
1952-08-29,0.980578,0.959750,0.937819,0.921458,0.895717
1952-09-30,0.980312,0.959098,0.937551,0.920961,0.891676
1952-10-31,0.981108,0.959745,0.937583,0.919290,0.897248
...,...,...,...,...,...
2021-06-30,0.999109,0.995047,0.986270,0.972698,0.956067
2021-07-30,0.999360,0.996519,0.989701,0.978165,0.965719
2021-08-31,0.999096,0.995720,0.987648,0.975238,0.961628


In [7]:
yields.columns = [1, 2, 3, 4, 5]
yields.index.name = 'date'
yields.columns.name = 'maturity'
yields

maturity,1,2,3,4,5
date,,,,,
1952-06-30,0.018390,0.019094,0.019751,0.019128,0.020167
1952-07-31,0.019236,0.020275,0.020907,0.019994,0.021390
1952-08-29,0.019506,0.020485,0.021360,0.020408,0.022002
1952-09-30,0.019885,0.020881,0.021495,0.020598,0.022918
1952-10-31,0.019125,0.020600,0.021483,0.021024,0.021673
...,...,...,...,...,...
2021-06-30,0.000892,0.002483,0.004613,0.006916,0.008980
2021-07-30,0.000642,0.001741,0.003445,0.005512,0.006969
2021-08-31,0.000904,0.002145,0.004143,0.006273,0.007821


# Save to Excel

In [8]:
with pd.ExcelWriter('treasury_data.xlsx') as writer:  
    prices.to_excel(writer, sheet_name= 'prices')
    yields.to_excel(writer, sheet_name='yields')